In [1]:
import xarray as xr
import requests
import netCDF4
import boto3
import boto3
from botocore import UNSIGNED
from botocore.config import Config
import matplotlib.pyplot as plt

In [2]:
bucket_name = 'noaa-goes16'
product_name = 'ABI-L2-CMIPC'#'ABI-L2-CMIPC'#'ABI-L1b-RadF'
year = 2020
day_of_year = 79
hour = 14 
band = 5
#https://github.com/HamedAlemo/visualize-goes16/blob/master/visualize_GOES16_from_AWS.ipynb

In [3]:
# Initialize s3 client. 
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

In [4]:
def get_s3_keys(bucket, s3_client, prefix = ''):
    """
    Generate the keys in an S3 bucket.

    :param bucket: Name of the S3 bucket.
    :param prefix: Only fetch keys that start with this prefix (optional).
    """
    
    kwargs = {'Bucket': bucket}

    if isinstance(prefix, str):
        kwargs['Prefix'] = prefix

    while True:
        resp = s3_client.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                yield key

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break

In [5]:
'''
kwargs = {'Bucket': bucket_name}
kwargs['Prefix'] = f'{product_name}/{"2019"}/{day_of_year:03.0f}/{hour:02.0f}/OR_{product_name}-M3C{band:02.0f}'
while True:
        print('here')
        resp = s3_client.list_objects_v2(**kwargs)
        print(resp)
        for obj in resp['Contents']:
            key = obj['Key']
            if key.startswith(prefix):
                print('here')
                print( key )

        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
'''

'\nkwargs = {\'Bucket\': bucket_name}\nkwargs[\'Prefix\'] = f\'{product_name}/{"2019"}/{day_of_year:03.0f}/{hour:02.0f}/OR_{product_name}-M3C{band:02.0f}\'\nwhile True:\n        print(\'here\')\n        resp = s3_client.list_objects_v2(**kwargs)\n        print(resp)\n        for obj in resp[\'Contents\']:\n            key = obj[\'Key\']\n            if key.startswith(prefix):\n                print(\'here\')\n                print( key )\n\n        try:\n            kwargs[\'ContinuationToken\'] = resp[\'NextContinuationToken\']\n        except KeyError:\n            break\n'

In [6]:
keys = get_s3_keys(bucket_name,
                s3_client,
                prefix = f'{product_name}/{year}/{day_of_year:03.0f}/{hour:02.0f}/OR_{product_name}-M6C{band:02.0f}'
                )
#-M3C{band:02.0f}

In [7]:
key = [key for key in keys][0] # selecting the first measurement taken within the hour
key

'ABI-L2-CMIPC/2020/079/14/OR_ABI-L2-CMIPC-M6C05_G16_s20200791401145_e20200791403518_c20200791404016.nc'

In [8]:
(keys)

<generator object get_s3_keys at 0x0000020473E3A040>

In [9]:
#key = [key for key in keys][0] # selecting the first measurement taken within the hour

In [10]:
resp = requests.get(f'https://{bucket_name}.s3.amazonaws.com/{key}')

In [11]:
file_name = key.split('/')[-1].split('.')[0]
file_name

'OR_ABI-L2-CMIPC-M6C05_G16_s20200791401145_e20200791403518_c20200791404016'

In [12]:
nc4_ds = netCDF4.Dataset(file_name, memory = resp.content)

In [13]:
store = xr.backends.NetCDF4DataStore(nc4_ds)
DS = xr.open_dataset(store)
DS

<xarray.Dataset>
Dimensions:                                           (y: 3000, x: 5000, number_of_time_bounds: 2, number_of_image_bounds: 2, band: 1)
Coordinates:
    t                                                 datetime64[ns] ...
  * y                                                 (y) float32 0.1282 ... ...
  * x                                                 (x) float32 -0.1013 ......
    y_image                                           float32 ...
    x_image                                           float32 ...
    band_wavelength                                   (band) float32 ...
    band_id                                           (band) int32 ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds, band
Data variables: (12/33)
    CMI                                               (y, x) float32 ...
    DQF                                               (y, x) float32 ...
    time_bounds                                       (number_of_time_bounds) datetime64[ns] ...
    goes_imager_projection                            int32 ...
    y_image_bounds                                    (number_of_image_bounds) float32 ...
    x_image_bounds                                    (number_of_image_bounds) float32 ...
    ...                                                ...
    planck_bc1                                        float32 ...
    planck_bc2                                        float32 ...
    focal_plane_temperature_threshold_exceeded_count  float64 ...
    maximum_focal_plane_temperature                   float32 ...
    focal_plane_temperature_threshold_increasing      float32 ...
    focal_plane_temperature_threshold_decreasing      float32 ...
Attributes: (12/29)
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    Metadata_Conventions:      Unidata Dataset Discovery v1.0
    standard_name_vocabulary:  CF Standard Name Table (v35, 20 July 2016)
    institution:               DOC/NOAA/NESDIS > U.S. Department of Commerce,...
    project:                   GOES
    ...                        ...
    cdm_data_type:             Image
    time_coverage_start:       2020-03-19T14:01:14.5Z
    time_coverage_end:         2020-03-19T14:03:51.8Z
    timeline_id:               ABI Mode 6
    production_data_source:    Realtime
    id:                        8d056231-a7a7-4c39-b074-b7dfa635e53b

In [ ]:
fig = plt.figure(figsize=(12, 12))
plt.imshow(DS.CMI, cmap='gray') #DS.Rad
plt.axis('off')
plt.savefig(f'{file_name}.png', dpi=300, facecolor='w', edgecolor='w')